### Import libraries

In [1]:
import pandas as pd
import numpy as np

import talib as ta
import random

import warnings
warnings.filterwarnings("ignore")

from datetime import datetime
import time

### Download data

In [2]:
# import yfinance as yf

# tickers = ['AAPL', 'MSFT', 'GOOGL', 'AMZN', 'TSLA', 'META', 'NVDA']

# start_date = "2020-01-01"
# end_date = datetime.now().strftime("%Y-%m-%d")

# # Download historical data from yf API
# data = yf.download(tickers, start=start_date, end=end_date, group_by='ticker')

# # download data to csv using ticker, exchange, date range and "raw" indicator in the filename
# data.to_csv(f"data_{'_'.join(tickers)}_{start_date}_{end_date}_raw.csv")

In [3]:
# Load the CSV with MultiIndex columns (Tickers, OHLCV)
df = pd.read_csv("../00_data/data_AAPL_MSFT_GOOGL_AMZN_TSLA_META_NVDA_2020-01-01_2025-10-26_raw.csv", header=[0,1], index_col=0)

# Drop any rows that are completely NaN (e.g. 'Date' row)
df = df.dropna(how='all')

# Convert all values to float
df = df.astype(float)

# Show the result
df.head()

Ticker            MSFT                                                  \
Price             Open        High         Low       Close      Volume   
Date                                                                     
2020-01-02  151.040856  152.895807  150.612792  152.791168  22622100.0   
2020-01-03  150.603275  152.153817  150.355939  150.888641  21116200.0   
2020-01-06  149.423644  151.345190  148.881420  151.278595  20813700.0   
2020-01-07  151.554487  151.887418  149.651970  149.899292  21634100.0   
2020-01-08  151.183463  152.962326  150.251234  152.286926  27746500.0   

Ticker           AMZN                                               ...  \
Price            Open       High        Low      Close      Volume  ...   
Date                                                                ...   
2020-01-02  93.750000  94.900497  93.207497  94.900497  80580000.0  ...   
2020-01-03  93.224998  94.309998  93.224998  93.748497  75288000.0  ...   
2020-01-06  93.000000  95.184502  93.000000  95.143997  81236000.0  ...   
2020-01-07  95.224998  95.694504  94.601997  95.343002  80898000.0  ...   
2020-01-08  94.902000  95.550003  94.321999  94.598503  70160000.0  ...   

Ticker            META                                                  \
Price             Open        High         Low       Close      Volume   
Date                                                                     
2020-01-02  205.483256  208.504623  205.006201  208.494690  12077100.0   
2020-01-03  205.940433  209.110876  205.682017  207.391479  11188400.0   
2020-01-06  205.433564  211.476314  205.254674  211.297424  17058900.0   
2020-01-07  211.516088  213.265300  210.452637  211.754608  14912400.0   
2020-01-08  211.694968  214.915122  211.307358  213.901367  13475000.0   

Ticker           AAPL                                                
Price            Open       High        Low      Close       Volume  
Date                                                                 
2020-01-02  71.545890  72.598892  71.292304  72.538513  135480400.0  
2020-01-03  71.765667  72.594055  71.608685  71.833290  146322800.0  
2020-01-06  70.954203  72.444336  70.703027  72.405693  118387200.0  
2020-01-07  72.415337  72.671341  71.845369  72.065147  108872000.0  
2020-01-08  71.768101  73.526318  71.768101  73.224426  132079200.0  

[5 rows x 35 columns]

### Ticker selection

In [4]:
# # random
# # randomly shuffle the level 0 of the columns (the tickers), to select one random tickers for strategy and becktesting

# ticker_ = random.choice(df.columns.levels[0])

# print(f"Randomly selected ticker for strategy and backtesting: {random_ticker}")

# fixed
# fixed ticker for strategy and backtesting
ticker_ = 'AAPL'

### Data cleaning

In [5]:
# clean df to keep only the selected ticker
df = df.xs(ticker_, level=0, axis=1)

# lowercase columns
df.columns = [col.lower() for col in df.columns]

# set index to datetime
df.index = pd.to_datetime(df.index)

# rename index to date
df.index.name = 'date'

# show df
df.head()

,open,high,low,close,volume
date,,,,,
2020-01-02,71.545890,72.598892,71.292304,72.538513,135480400.0
2020-01-03,71.765667,72.594055,71.608685,71.833290,146322800.0
2020-01-06,70.954203,72.444336,70.703027,72.405693,118387200.0
2020-01-07,72.415337,72.671341,71.845369,72.065147,108872000.0
2020-01-08,71.768101,73.526318,71.768101,73.224426,132079200.0


### Add Indicators

In [6]:
# calculate VWAP
def calculate_vwap(df):
    """Calculate VWAP for the given DataFrame."""
    vwap = (df['volume'] * (df['high'] + df['low'] + df['close']) / 3).cumsum() / df['volume'].cumsum()
    return vwap

In [7]:
# calculate ATR for stop loss and take profit
sl_multiplier = 1.5
tp_multiplier = 3.0

def calculate_atr(df, period=14):
    """Calculate ATR for the given DataFrame."""
    high_low = df['high'] - df['low']
    high_close = (df['high'] - df['close'].shift()).abs()
    low_close = (df['low'] - df['close'].shift()).abs()
    true_range = pd.concat([high_low, high_close, low_close], axis=1).max(axis=1)
    atr = true_range.rolling(window=period).mean()
    return atr

"""
stop_loss = entry_price - (atr * sl_multiplier)
take_profit = entry_price + (atr * tp_multiplier)
"""

'\nstop_loss = entry_price - (atr * sl_multiplier)\ntake_profit = entry_price + (atr * tp_multiplier)\n'

In [8]:
# # Define function to add indicators
# indicators = ['BBANDS', 'DEMA', 'EMA', 'KAMA', 'MA',
#               'SAR', 'MACD', 'MFI', 'MOM', 'ROC',
#               'ROCP', 'STOCH', 'RSI', 'WILLR',
#               'WMA', 'STDDEV', 'VAR', 'SMA', 'PPO'
#               ]

In [9]:
# Define Candle Pattern Conditions
candles = [
    # Bullish reversal patterns
    'CDLHAMMER',                  # Hammer
    'CDLINVERTEDHAMMER',          # Inverted Hammer
    'CDLMORNINGSTAR',             # Morning Star
    'CDLMORNINGDOJISTAR',         # Morning Doji Star
    'CDLENGULFING',               # Bullish Engulfing
    'CDLPIERCING',                # Piercing Pattern
    'CDLHARAMI',                  # Bullish Harami
    'CDLHARAMICROSS',             # Bullish Harami Cross
    'CDLTAKURI',                  # Takuri (Dragonfly Doji)
    
    # Bullish continuation patterns
    'CDL3WHITESOLDIERS',          # Three White Soldiers
    'CDLRISEFALL3METHODS',        # Rising Three Methods
    'CDLMATHOLD',                 # Mat Hold
    'CDLSEPARATINGLINES',         # Bullish Separating Lines
    'CDLTASUKIGAP',               # Bullish Tasuki Gap uptrend
    
    # Bullish bottom patterns
    'CDLABANDONEDBABY',           # Abandoned Baby
    'CDLLADDERBOTTOM',            # Ladder Bottom
    'CDLMATCHINGLOW',             # Matching Low
    'CDLUNIQUE3RIVER',            # Unique Three River
    
    # Bullish special patterns
    'CDL3INSIDE',                 # Three Inside Up
    'CDL3OUTSIDE',                # Three Outside Up
    'CDLBELTHOLD',                # Belt Hold
    'CDLBREAKAWAY',               # Breakaway
    'CDLKICKING',                 # Kicking
    'CDLKICKINGBYLENGTH',         # Kicking By Length
    'CDLSTICKSANDWICH'            # Stick Sandwich
]


In [10]:
# TWAP
# ....

### Indicators Selection

In [11]:
# Indicators (features) are splitted in two groups: indicators (list) and candles (list) 

# indicators list is already defined above
# candles list is already defined above

# First we will implement the candle patterns as features, then the indicators

In [12]:
# # interate over candles and calculate each pattern
# for candle in candles:
#     df[candle] = getattr(ta, candle)(df['open'], df['high'], df['low'], df['close'])
#     print(f"Calculated {candle}")

In [13]:
df.head()

,open,high,low,close,volume
date,,,,,
2020-01-02,71.545890,72.598892,71.292304,72.538513,135480400.0
2020-01-03,71.765667,72.594055,71.608685,71.833290,146322800.0
2020-01-06,70.954203,72.444336,70.703027,72.405693,118387200.0
2020-01-07,72.415337,72.671341,71.845369,72.065147,108872000.0
2020-01-08,71.768101,73.526318,71.768101,73.224426,132079200.0


In [14]:
# select random indicator or candlestick pattern

In [15]:
# Add necessary imports for backtesting
from backtesting import Strategy, Backtest
from backtesting.lib import crossover

# Strategy implementation using your existing functions and data
class VWAPEngulfingStrategy(Strategy):
    """
    VWAP + Bullish Engulfing Strategy with ATR-based Stop Loss and Take Profit
    
    Entry Conditions:
    (no crossover, price simply above VWAP)!
    - Price is above VWAP (bullish signal)
    - Bullish Engulfing pattern detected
    
    Exit Conditions:
    - Stop Loss: entry_price - (atr * sl_multiplier)
    - Take Profit: entry_price + (atr * tp_multiplier)
    """
    
    # Strategy parameters (using your existing values)
    atr_period = 14
    sl_multiplier = 1.5
    tp_multiplier = 3.0
    
    def init(self):
        """Initialize indicators using your existing functions"""
        # Calculate VWAP using your calculate_vwap function logic
        typical_price = (self.data.High + self.data.Low + self.data.Close) / 3
        volume_price = typical_price * self.data.Volume
        self.vwap = self.I(lambda: volume_price.cumsum() / self.data.Volume.cumsum(), name='VWAP')
        
        # Calculate ATR using talib (as in your notebook)
        self.atr = self.I(ta.ATR, self.data.High, self.data.Low, self.data.Close, self.atr_period, name='ATR')
        
        # Calculate Bullish Engulfing pattern using talib
        self.engulfing = self.I(ta.CDLENGULFING, self.data.Open, self.data.High, 
                               self.data.Low, self.data.Close, name='Engulfing')
        
        # Track entry levels
        self.entry_price = None
        self.stop_loss_level = None
        self.take_profit_level = None
        
    def next(self):
        """Define trading logic"""
        # Entry conditions
        if (not self.position and 
            self.data.Close[-1] > self.vwap[-1] and  # Price above VWAP
            self.engulfing[-1] > 0):  # Bullish engulfing pattern (100)
            
            # Enter long position
            self.buy()
            
            # Set entry price and levels using your multipliers
            self.entry_price = self.data.Close[-1]
            self.stop_loss_level = self.entry_price - (self.atr[-1] * self.sl_multiplier)
            self.take_profit_level = self.entry_price + (self.atr[-1] * self.tp_multiplier)
        
        # Exit conditions
        if self.position:
            # Stop Loss
            if self.data.Low[-1] <= self.stop_loss_level:
                self.position.close()
                self.entry_price = None
                
            # Take Profit
            elif self.data.High[-1] >= self.take_profit_level:
                self.position.close()
                self.entry_price = None

# Prepare data for backtesting (ensure proper column names)
# The backtesting library expects uppercase column names
df_backtest = df.copy()
df_backtest.columns = ['Open', 'High', 'Low', 'Close', 'Volume']

# Run backtest on your actual data
bt = Backtest(df_backtest, VWAPEngulfingStrategy, 
              cash=10000, commission=0.002)

# Execute backtest
results = bt.run()

# Display results
print("="*60)
print("VWAP + BULLISH ENGULFING STRATEGY BACKTEST RESULTS")
print(f"Ticker: {ticker_}")
print(f"Period: {df.index[0].strftime('%Y-%m-%d')} to {df.index[-1].strftime('%Y-%m-%d')}")
print("="*60)

# Key performance metrics
key_metrics = [
    'Return [%]',
    'Buy & Hold Return [%]',
    'Max Drawdown [%]',
    'Volatility (ann.) [%]',
    'Sharpe Ratio',
    '# Trades',
    'Win Rate [%]',
    'Best Trade [%]',
    'Worst Trade [%]',
    'Avg Trade [%]'
]

for metric in key_metrics:
    if metric in results.index:
        value = results[metric]
        if isinstance(value, (int, float)):
            if '%' in metric:
                print(f"{metric:<25}: {value:>8.2f}%")
            elif metric == '# Trades':
                print(f"{metric:<25}: {value:>8.0f}")
            else:
                print(f"{metric:<25}: {value:>8.4f}")

print(f"\nStrategy Parameters:")
print(f"{'ATR Period':<25}: {VWAPEngulfingStrategy.atr_period}")
print(f"{'Stop Loss Multiplier':<25}: {VWAPEngulfingStrategy.sl_multiplier}")
print(f"{'Take Profit Multiplier':<25}: {VWAPEngulfingStrategy.tp_multiplier}")

Loading BokehJS ...

VWAP + BULLISH ENGULFING STRATEGY BACKTEST RESULTS
Ticker: AAPL
Period: 2020-01-02 to 2025-10-24
Return [%]               :    22.41%
Buy & Hold Return [%]    :   240.89%
Sharpe Ratio             :   0.2371
# Trades                 :       31
Win Rate [%]             :    51.61%
Best Trade [%]           :    22.07%
Worst Trade [%]          :   -12.91%

Strategy Parameters:
ATR Period               : 14
Stop Loss Multiplier     : 1.5
Take Profit Multiplier   : 3.0


In [16]:
# backtest out of sample data

In [17]:
# create performance report with the backtest results for all strategies after out of sample backtest

In [18]:
# optimize the parameters of the 3 best strategies
"""
def optimize
(
self,
*,
maximize='SQN',
method='grid',
max_tries=None,
constraint=None,
return_heatmap=False,
return_optimization=False,
random_state=None,
**kwargs)
Optimize strategy parameters to an optimal combination. Returns result pd.Series of the best run.

maximize is a string key from the Backtest.run()-returned results series, or a function that accepts this series object and returns a number; the higher the better. By default, the method maximizes Van Tharp's System Quality Number.

method is the optimization method. Currently two methods are supported:

"grid" which does an exhaustive (or randomized) search over the cartesian product of parameter combinations, and
"sambo" which finds close-to-optimal strategy parameters using model-based optimization, making at most max_tries evaluations.
max_tries is the maximal number of strategy runs to perform. If method="grid", this results in randomized grid search. If max_tries is a floating value between (0, 1], this sets the number of runs to approximately that fraction of full grid space. Alternatively, if integer, it denotes the absolute maximum number of evaluations. If unspecified (default), grid search is exhaustive, whereas for method="sambo", max_tries is set to 200.

constraint is a function that accepts a dict-like object of parameters (with values) and returns True when the combination is admissible to test with. By default, any parameters combination is considered admissible.

If return_heatmap is True, besides returning the result series, an additional pd.Series is returned with a multiindex of all admissible parameter combinations, which can be further inspected or projected onto 2D to plot a heatmap (see plot_heatmaps()).

If return_optimization is True and method = 'sambo', in addition to result series (and maybe heatmap), return raw scipy.optimize.OptimizeResult for further inspection, e.g. with SAMBO's plotting tools.

If you want reproducible optimization results, set random_state to a fixed integer random seed.

Additional keyword arguments represent strategy arguments with list-like collections of possible values. For example, the following code finds and returns the "best" of the 7 admissible (of the 9 possible) parameter combinations:

best_stats = backtest.optimize(sma1=[5, 10, 15], sma2=[10, 20, 40],
                               constraint=lambda p: p.sma1 < p.sma2)
"""

'\ndef optimize\n(\nself,\n*,\nmaximize=\'SQN\',\nmethod=\'grid\',\nmax_tries=None,\nconstraint=None,\nreturn_heatmap=False,\nreturn_optimization=False,\nrandom_state=None,\n**kwargs)\nOptimize strategy parameters to an optimal combination. Returns result pd.Series of the best run.\n\nmaximize is a string key from the Backtest.run()-returned results series, or a function that accepts this series object and returns a number; the higher the better. By default, the method maximizes Van Tharp\'s System Quality Number.\n\nmethod is the optimization method. Currently two methods are supported:\n\n"grid" which does an exhaustive (or randomized) search over the cartesian product of parameter combinations, and\n"sambo" which finds close-to-optimal strategy parameters using model-based optimization, making at most max_tries evaluations.\nmax_tries is the maximal number of strategy runs to perform. If method="grid", this results in randomized grid search. If max_tries is a floating value between 

In [19]:
# 3 best strategies after optimization should be tested on 1 hour data

In [20]:
# plot the strategies
"""
def plot
(
self,
*,
results=None,
filename=None,
plot_width=None,
plot_equity=True,
plot_return=False,
plot_pl=True,
plot_volume=True,
plot_drawdown=False,
plot_trades=True,
smooth_equity=False,
relative_equity=True,
superimpose=True,
resample=True,
reverse_indicators=False,
show_legend=True,
open_browser=True)
Plot the progression of the last backtest run.

If results is provided, it should be a particular result pd.Series such as returned by Backtest.run() or Backtest.optimize(), otherwise the last run's results are used.

filename is the path to save the interactive HTML plot to. By default, a strategy/parameter-dependent file is created in the current working directory.

plot_width is the width of the plot in pixels. If None (default), the plot is made to span 100% of browser width. The height is currently non-adjustable.

If plot_equity is True, the resulting plot will contain an equity (initial cash plus assets) graph section. This is the same as plot_return plus initial 100%.

If plot_return is True, the resulting plot will contain a cumulative return graph section. This is the same as plot_equity minus initial 100%.

If plot_pl is True, the resulting plot will contain a profit/loss (P/L) indicator section.

If plot_volume is True, the resulting plot will contain a trade volume section.

If plot_drawdown is True, the resulting plot will contain a separate drawdown graph section.

If plot_trades is True, the stretches between trade entries and trade exits are marked by hash-marked tractor beams.

If smooth_equity is True, the equity graph will be interpolated between fixed points at trade closing times, unaffected by any interim asset volatility.

If relative_equity is True, scale and label equity graph axis with return percent, not absolute cash-equivalent values.

If superimpose is True, superimpose larger-timeframe candlesticks over the original candlestick chart. Default downsampling rule is: monthly for daily data, daily for hourly data, hourly for minute data, and minute for (sub-)second data. superimpose can also be a valid Pandas offset string, such as '5T' or '5min', in which case this frequency will be used to superimpose. Note, this only works for data with a datetime index.

If resample is True, the OHLC data is resampled in a way that makes the upper number of candles for Bokeh to plot limited to 10_000. This may, in situations of overabundant data, improve plot's interactive performance and avoid browser's Javascript Error: Maximum call stack size exceeded or similar. Equity & dropdown curves and individual trades data is, likewise, reasonably aggregated. resample can also be a Pandas offset string, such as '5T' or '5min', in which case this frequency will be used to resample, overriding above numeric limitation. Note, all this only works for data with a datetime index.

If reverse_indicators is True, the indicators below the OHLC chart are plotted in reverse order of declaration.

If show_legend is True, the resulting plot graphs will contain labeled legends.

If open_browser is True, the resulting filename will be opened in the default web browser.

def run
(
self, **kwargs)
Run the backtest. Returns pd.Series with results and statistics.

Keyword arguments are interpreted as strategy parameters.
"""

"\ndef plot\n(\nself,\n*,\nresults=None,\nfilename=None,\nplot_width=None,\nplot_equity=True,\nplot_return=False,\nplot_pl=True,\nplot_volume=True,\nplot_drawdown=False,\nplot_trades=True,\nsmooth_equity=False,\nrelative_equity=True,\nsuperimpose=True,\nresample=True,\nreverse_indicators=False,\nshow_legend=True,\nopen_browser=True)\nPlot the progression of the last backtest run.\n\nIf results is provided, it should be a particular result pd.Series such as returned by Backtest.run() or Backtest.optimize(), otherwise the last run's results are used.\n\nfilename is the path to save the interactive HTML plot to. By default, a strategy/parameter-dependent file is created in the current working directory.\n\nplot_width is the width of the plot in pixels. If None (default), the plot is made to span 100% of browser width. The height is currently non-adjustable.\n\nIf plot_equity is True, the resulting plot will contain an equity (initial cash plus assets) graph section. This is the same as pl

In [21]:
# Monte Carlo simulation or bootstrap to evaluate the robustness of the strategies
"""
def random_ohlc_data
(
example_data, *, frac=1.0, random_state=None)
OHLC data generator. The generated OHLC data has basic descriptive statistics similar to the provided example_data.

frac is a fraction of data to sample (with replacement). Values greater than 1 result in oversampling.

Such random data can be effectively used for stress testing trading strategy robustness, Monte Carlo simulations, significance testing, etc.

>>> from backtesting.test import EURUSD
>>> ohlc_generator = random_ohlc_data(EURUSD)
>>> next(ohlc_generator)  # returns new random data
...
>>> next(ohlc_generator)  # returns new random data
...
def resample_apply
(
rule, func, series, *args, agg=None, **kwargs)
Apply func (such as an indicator) to series, resampled to a time frame specified by rule. When called from inside Strategy.init(), the result (returned) series will be automatically wrapped in Strategy.I() wrapper method.

rule is a valid Pandas offset string indicating a time frame to resample series to.

func is the indicator function to apply on the resampled series.

series is a data series (or array), such as any of the Strategy.data series. Due to pandas resampling limitations, this only works when input series has a datetime index.

agg is the aggregation function to use on resampled groups of data. Valid values are anything accepted by pandas/resample/.agg(). Default value for dataframe input is OHLCV_AGG dictionary. Default value for series input is the appropriate entry from OHLCV_AGG if series has a matching name, or otherwise the value "last", which is suitable for closing prices, but you might prefer another (e.g. "max" for peaks, or similar).

Finally, any *args and **kwargs that are not already eaten by implicit Strategy.I() call are passed to func.

For example, if we have a typical moving average function SMA(values, lookback_period), hourly data source, and need to apply the moving average MA(10) on a daily time frame, but don't want to plot the resulting indicator, we can do:

class System(Strategy):
    def init(self):
        self.sma = resample_apply(
            'D', SMA, self.data.Close, 10, plot=False)
The above short snippet is roughly equivalent to:

class System(Strategy):
    def init(self):
        # Strategy exposes <code>self.data</code> as raw NumPy arrays.
        # Let's convert closing prices back to pandas Series.
        close = self.data.Close.s

        # Resample to daily resolution. Aggregate groups
        # using their last value (i.e. closing price at the end
        # of the day). Notice `label='right'`. If it were set to
        # 'left' (default), the strategy would exhibit
        # look-ahead bias.
        daily = close.resample('D', label='right').agg('last')

        # We apply SMA(10) to daily close prices,
        # then reindex it back to original hourly index,
        # forward-filling the missing values in each day.
        # We make a separate function that returns the final
        # indicator array.
        def SMA(series, n):
            from backtesting.test import SMA
            return SMA(series, n).reindex(close.index).ffill()

        # The result equivalent to the short example above:
        self.sma = self.I(SMA, daily, 10, plot=False)
    """

'\ndef random_ohlc_data\n(\nexample_data, *, frac=1.0, random_state=None)\nOHLC data generator. The generated OHLC data has basic descriptive statistics similar to the provided example_data.\n\nfrac is a fraction of data to sample (with replacement). Values greater than 1 result in oversampling.\n\nSuch random data can be effectively used for stress testing trading strategy robustness, Monte Carlo simulations, significance testing, etc.\n\n>>> from backtesting.test import EURUSD\n>>> ohlc_generator = random_ohlc_data(EURUSD)\n>>> next(ohlc_generator)  # returns new random data\n...\n>>> next(ohlc_generator)  # returns new random data\n...\ndef resample_apply\n(\nrule, func, series, *args, agg=None, **kwargs)\nApply func (such as an indicator) to series, resampled to a time frame specified by rule. When called from inside Strategy.init(), the result (returned) series will be automatically wrapped in Strategy.I() wrapper method.\n\nrule is a valid Pandas offset string indicating a time f

In [22]:
"""
class MultiBacktest
(
df_list, strategy_cls, **kwargs)
Multi-dataset Backtest wrapper.

Run supplied Strategy on several instruments, in parallel. Used for comparing strategy runs across many instruments or classes of instruments. Example:

from backtesting.test import EURUSD, BTCUSD, SmaCross
btm = MultiBacktest([EURUSD, BTCUSD], SmaCross)
stats_per_ticker: pd.DataFrame = btm.run(fast=10, slow=20)
heatmap_per_ticker: pd.DataFrame = btm.optimize(...)
Methods
def optimize
(
self, **kwargs)
Wraps Backtest.optimize(), but returns pd.DataFrame with currency indexes in columns.

heamap: pd.DataFrame = btm.optimize(...)
from backtesting.plot import plot_heatmaps
plot_heatmaps(heatmap.mean(axis=1))
def run
(
self, **kwargs)
Wraps Backtest.run(). Returns pd.DataFrame with currency indexes in columns.
"""

'\nclass MultiBacktest\n(\ndf_list, strategy_cls, **kwargs)\nMulti-dataset Backtest wrapper.\n\nRun supplied Strategy on several instruments, in parallel. Used for comparing strategy runs across many instruments or classes of instruments. Example:\n\nfrom backtesting.test import EURUSD, BTCUSD, SmaCross\nbtm = MultiBacktest([EURUSD, BTCUSD], SmaCross)\nstats_per_ticker: pd.DataFrame = btm.run(fast=10, slow=20)\nheatmap_per_ticker: pd.DataFrame = btm.optimize(...)\nMethods\ndef optimize\n(\nself, **kwargs)\nWraps Backtest.optimize(), but returns pd.DataFrame with currency indexes in columns.\n\nheamap: pd.DataFrame = btm.optimize(...)\nfrom backtesting.plot import plot_heatmaps\nplot_heatmaps(heatmap.mean(axis=1))\ndef run\n(\nself, **kwargs)\nWraps Backtest.run(). Returns pd.DataFrame with currency indexes in columns.\n'

In [23]:
# Final performance report